In [2]:
import paramiko
import os
import datetime

In [3]:
host = "107.191.32.220" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "juba-jc1" #hard-coded
username = "joann" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [4]:
remote_folder="/mnt/drv5/joann/Census and EASI/"

EASI_A="/home/jian/Docs/EASI/2018-07-19/ZIP4_18_DATA_A2_CSV/ZIP4_18_DATA_A2.CSV"
EASI_B="/home/jian/Docs/EASI/2018-07-19/ZIP4_18_DATA_B2_CSV/ZIP4_18_DATA_B2.CSV"
EASI_C="/home/jian/Docs/EASI/2018-07-19/ZIP4_18_DATA_C2_CSV/ZIP4_18_DATA_C2.CSV"
list_local_files=[EASI_A,EASI_B,EASI_C]
for local_file in list_local_files:
    sftp.put(EASI_A,remote_folder+os.path.basename(local_file))
sftp.close()
transport.close()

In [7]:
import paramiko
import os

class MySFTPClient(paramiko.SFTPClient):
    def put_dir(self, source, target):
        ''' Uploads the contents of the source directory to the target path. The
            target directory needs to exists. All subdirectories in source are 
            created under target.
        '''
        for item in os.listdir(source):
            if os.path.isfile(os.path.join(source, item)):
                self.put(os.path.join(source, item), '%s/%s' % (target, item))
            else:
                self.mkdir('%s/%s' % (target, item), ignore_existing=True)
                self.put_dir(os.path.join(source, item), '%s/%s' % (target, item))

    def mkdir(self, path, mode=511, ignore_existing=False):
        ''' Augments mkdir by adding an option to not fail if the folder exists  '''
        try:
            super(MySFTPClient, self).mkdir(path, mode)
        except IOError:
            if ignore_existing:
                pass
            else:
                raise

In [9]:
source_path

'/home/jian/Docs/Household_and_Population/Cencus_2019/'

In [8]:
transport = paramiko.Transport((host, port))
transport.connect(username=username, password=password)
sftp = MySFTPClient.from_transport(transport)

target_path="/mnt/drv5/joann/Census and EASI/census_19"
sftp.mkdir(target_path, ignore_existing=True)
sftp.put_dir(source_path, target_path)
sftp.close()